# Compare the data produced by IMAGER and CASA

This notebook compares the data cubes produced by IMAGER and CASA. 

## Initialization

In [ ]:
!pip install -r requirements.txt > /dev/null

In [ ]:
import numpy as np
from astropy import units as u
from spectral_cube import SpectralCube
from matplotlib import pyplot as plt
from compass import datadir, extract_spectrum_from_cube, reference_position
from compass.cubes import extract_line, zeroth_order_moment

In [ ]:
%matplotlib widget
%load_ext jupyter_black

## Define the source, the frequency setting and the spectral window

In [ ]:
source = "bhr71"
pos = reference_position[source]
setting = 6
casa_settings = {1: "a", 2: "b", 3: "c", 4: "d", 5: "e", 6: "f", 7: "g", 8: "h", 9: "i"}
spw = 25

## Read the cube produced by CASA and IMAGER

In [ ]:
cube_casa = SpectralCube.read(
    datadir
    + "../reduced-casa/{}/tune{}/oussid.s8_0.BHR71-IRS1_sci.spw{}.cube.selfcal.I.iter1.image.fits".format(
        source, casa_settings[setting], spw
    )
).with_spectral_unit(u.GHz)

In [ ]:
cube_imager = SpectralCube.read(
    datadir + "bhr71/cubes/bhr71-set{}-spw{}-lines.fits".format(setting, spw)
).with_spectral_unit(u.GHz)

## Make a figure to compare the spectra extracted at a given position

Extract spectra from the CASA and IMAGER cubes:

In [ ]:
spectrum_casa = extract_spectrum_from_cube(cube_casa, pos)
spectrum_imager = extract_spectrum_from_cube(cube_imager, pos)

Plot the spectra:

In [ ]:
plt.rcParams["font.family"] = "serif"
fig, ax = plt.subplots(
    num="BHR71 setting {} spw {}".format(setting, spw), figsize=(12, 6), clear=True
)
i = ax.step(
    spectrum_imager.spectral_axis.value,
    spectrum_imager.to(u.K).value,
    label="Imager",
    zorder=2,
)
c = ax.step(
    spectrum_casa.spectral_axis.value,
    spectrum_casa.to(u.K).value,
    label="Casa",
    zorder=1,
)
baseline = np.zeros_like(spectrum_imager.spectral_axis.value)
b = ax.plot(
    spectrum_imager.spectral_axis.value,
    baseline,
    color="black",
    linestyle="dotted",
)
ax.set_xlabel("Frequency (GHz)")
ax.set_ylabel("Brightness temperature (K)")
ax.set_xlim(297.70, 297.90)
ll = ax.legend()

Overplot the RMS:

In [ ]:
show_rms = True
restfreq_imager = cube_imager.header["RESTFRQ"] * u.Hz
if show_rms:
    rms = (cube_imager.header["NOISEMEA"] * cube_imager.unit).to(
        u.K, u.brightness_temperature(restfreq_imager, cube_imager.beam)
    )
    ax.axhspan(
        -rms.value,
        rms.value,
        color="grey",
        alpha=0.25,
    )  # +/- 1 sigma
    ax.axhspan(
        -rms.value * 3,
        rms.value * 3,
        color="grey",
        alpha=0.10,
    )  # +/- 3 sigma

Save the figure:

In [ ]:
plt.savefig("bhr71-spectra-imager-casa.pdf", bbox_inches="tight")

## Make a figure to compare the moment 0 maps of a line

Define the line frequency and velocity range:

In [ ]:
line_freq = 297.830091 * u.GHz
vmin, vmax = -5 * u.km / u.s, 10 * u.km / u.s

Extract the line from the cube:

In [ ]:
line_cube_imager = extract_line(cube_imager, restfreq=line_freq, vmin=vmin, vmax=vmax)
line_cube_casa = extract_line(cube_casa, restfreq=line_freq, vmin=vmin, vmax=vmax)

Check that the velocity range covers the entire line profile:

In [ ]:
line_spectrum = extract_spectrum_from_cube(line_cube_imager, pos)

In [ ]:
fig, ax = plt.subplots(num="Line profile".format(), figsize=(8, 5), clear=True)
i = ax.step(
    line_spectrum.spectral_axis.value,
    line_spectrum.to(u.K).value,
    label="Imager",
    zorder=2,
)
ax.set_xlabel("$v_\\mathrm{LSR}$ (km/s)")
ax.set_ylabel("Brightness temperature (K)")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 5), clear=True)
levels = np.linspace(10, 50, 5)  # * u.K * u.km / u.s
vmin, vmax = 0, 50

ax1, cb1 = zeroth_order_moment(
    line_cube_imager.to(u.K),
    fig,
    map_center=pos,
    map_size=3 * u.arcsec,
    vmin=vmin,
    vmax=vmax,
    levels=levels,
    subplot=121,
)

ax2, cb2 = zeroth_order_moment(
    line_cube_casa.to(u.K),
    fig,
    map_center=pos,
    map_size=3 * u.arcsec,
    vmin=vmin,
    vmax=vmax,
    levels=levels,
    subplot=122,
)

ax1.set_title("Imager", pad=10)
ra, dec = ax1.coords[0], ax1.coords[1]
ra.set_axislabel("RA (ICRS)")
dec.set_axislabel("Dec. (ICRS)")

ax2.set_title("Casa", pad=10)
ra, dec = ax2.coords[0], ax2.coords[1]
ra.set_axislabel(" ")
dec.set_axislabel(" ")
ra.set_ticks_visible(True)
ra.set_ticklabel_visible(True)
dec.set_ticks_visible(True)
dec.set_ticklabel_visible(False)

cb2.set_label("Zeroth-order moment (K km/s)", rotation=-90, labelpad=20)

Save the figure:

In [ ]:
plt.savefig("bhr71-moment0-imager-casa.pdf", bbox_inches="tight")